# Ταξινόμηση του Fashion-MNIST

Τώρα ήρθε η ώρα να εκπαιδεύσουμε ένα νευρωνικό δίκτυο σε ένα άλλο dataset. Θα χρησιμοποιήσουμε το [Fashion-MNIST dataset](https://github.com/zalandoresearch/fashion-mnist), αντί του κλασσικού MNIST dataset. Το κλασσικό MNIST για την ακρίβεια είναι ενα πολυ απλό dataset στο οποίο μπορούμε εύκολα να επιτύχουμε με τη βοήθεια των νευρωνικών δικτύων ακρίβεια πανω απο 97%. Το Fashion-MNIST είναι ένα σετ απο 28x28 εικόνες, απόχρωσης του γκρι, οι οποίες απεικονίζουν ρούχα. Είναι ένα πιο πολύπλοκο σετ απο το κλασσικό MNIST, με αποτέλεσμα να ειναι και πιο εμφανής η πραγματική απόδοση ενός δικτύου, αλλά είναι επίσης και μία καλύτερη εκδοχή των δεδομένων εκπαίδευσης που θα χρησιμοποιήσετε σε πραγματικές εφαρμογές.

<img src='assets/fashion-mnist-sprite.png' width=500px>

Σε αυτό το notebook, θα δημιουργήσετε το δικό σας νευρωνικό δίκτυο. Τα περισσότερα μέρη του κώδικα ειναι ίδια με αυτά που χρησιμοποιήσαμε στην προηγούμενη ενότητα, αλλά σκοπός είναι να γράψουμε το κώδικα απο μόνοι μας για να μάθουμε. Φυσικά, μπορείτε να συμβουλευτείτε όλα τα προηγούμενα notebooks καθώς προχωράτε στο κώδικα.

Πρώτα, φορτώνουμε το dataset μεσω της torchvision.

In [ ]:
import torch
from torchvision import datasets, transforms
import helper

# Καθόρισε ένα μετασχηματισμό για να κανονικοποιήσεις τα δεδομένα 
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Κατέβασε και φόρτωσε τα δεδομένα εκπαίδευσης
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Κατέβασε και φόρτωσε τα δεδομένα ελέγχου
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

Εδώ μπορούμε να δούμε μία απο τις εικόνες.

In [ ]:
image, label = next(iter(trainloader))
helper.imshow(image[0,:]);

## Χτίζοντας το δίκτυο

Σε αυτό το σημείο πρέπει να καθορίσουμε την αρχιτεκτονική του δικτύου μας. Όπως και στο κλασσικό MNIST, κάθε εικόνα έχει ανάλυση 28x28 με συνολικά 784 εικονοστοιχεία, και υπάρχουν 10 κλάσεις. Πρέπει να συμπεριλάβεται τουλάχιστον ένα κρυφό επίπεδο. Προτείνεται η χρήση της συνάρτησης ενεργοποίησης ReLU για τα επίπεδα και να υπολογίζετε τα logits ή το log-softmax απο το κάθε βήμα προσω τροφοδότησης. Απο εσάς εξαρτάται το πόσα επίπεδα θα χρησιμοποιήσετε και το πόσους κόμβους θα έχουν αυτά τα επίπεδα.

In [ ]:
from torch import nn, optim
import torch.nn.functional as F

In [ ]:
# Συμπληρώστε: Καθορίστε την αρχιτεκτονική του δικτύου σας εδω
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
    def forward(self, x):
        # γραμμικοποιήστε το τανυστή εισόδου
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x

# Εκπαιδεύστε το δίκτυο

Τώρα πρέπει να δημιουργήσετε το δίκτυο και να το εκπαιδεύσετε. Πρώτα πρέπει να καθορίσετε το κρίτηριο [the criterion](http://pytorch.org/docs/master/nn.html#loss-functions) (χρήση του `nn.CrossEntropyLoss` ή `nn.NLLLoss`) και του optimizer [the optimizer](http://pytorch.org/docs/master/optim.html) (συνήθως `optim.SGD` ή `optim.Adam`).

Μετά γράψτε το κώδικα εκπαίδευσης. Θυμηθείτε οτι η εκπαίδευση είναι μία ξεκάθαρη διεργασία μίας αλληλουχίας βημάτων:

* Εκτελώ πρόσω τροφοδότηση και υπολογίζω την έξοδο (logits) 
* Χρησιμοποιώ την έξδοδο (logits) για να υπολογίσω το σφάλμα/απώλεια (loss)
* Εκτελώ ένα βήμα προς τα πίσω (backward pass) στο δίκτυο με `loss.backward()` για να υπολογίσω τις μερικές παραγώγους
* Ανανέωση των βαρών και πολώσεων με τον optimizer

Καθορίζοντας σωστά τις υπερπαραμέτρους (κρυφά επίπεδα, learning rate, κτλ), μπορείτε να επιτύχετε ένα training loss κάτω απο 0.4.

In [ ]:
# Συμπληρώστε: Δημιουργήστε το δίκτυο, καθορίστε το κρίτήριο και τον optimizer
model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

In [ ]:
# Συμπληρώστε: Εκπαιδεύστε το δίκτυο σε αυτό το μπλοκ κώδικα
epochs = 5

for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Σφάλμα εκπαίδευσης: {running_loss/len(trainloader)}")

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import helper

# Ελέγξτε το δικτυό σας!

dataiter = iter(testloader)
images, labels = dataiter.next()
img = images[1]

# Συμπληρώστε: Υπολογίστε τις πιθανότητες για κάθε κλάση (softmax) για την img
ps = torch.exp(model(img))

# Απεικόνιστε την εικόνα και τις πιθανότητες
helper.view_classify(img, ps, version='Fashion')

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

Αυτό το notebook 📖 δημιουργήθηκε για το μάθημα ***Υπολογιστική Νοημοσύνη και Μηχανική Μάθηση*** του Τμήματος Μηχανικών Παραγωγής και Διοίκησης, της Πολυτεχνικής Σχολής του Δημοκριτείου Πανεπιστημίου Θράκης.<br>
This notebook is made available under the Creative Commons Attribution [(CC-BY)](https://creativecommons.org/licenses/by/4.0/legalcode) license. Code is also made available under the [MIT License](https://opensource.org/licenses/MIT).<br>
Author: Asst. Prof. Angelos Amanatiadis
<img src="assets/cc.png" style="width:55px; float: right; margin: 0px 0px 0px 0px;"></img>
<img src="assets/mit.png" style="width:40px; float: right; margin: 0px 10px 0px 0px;"></img>